In [1]:
import torch

# It is best to set device="cuda:0" each time a tensor is created
# instead of .to(device) with device as specified  
# accordingly:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Then, the data is created directly on the CUDA GPU 
# with no CPU (RAM) memory handling first.

# Convert from tensor on GPU to to numpy array on CPU.
#x_cpu = x.cpu().numpy()

# Check which device a tensor is on
#x.device

# Check if CUDA is available
#torch.cuda.is_available()

# CUDA device properties
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='GeForce RTX 2070', major=7, minor=5, total_memory=7951MB, multi_processor_count=36)

In [2]:
# edited ~/.spimagine
# colormap = grays

In [3]:
%gui qt5
import spimagine
from spimagine import volshow, volfig
import numpy as np
from scipy import signal
from glob import glob
from pydicom.filereader import dcmread

#"""
gaussian_kernel_width = 40
gaussian_std = 10

# Loading MRI volume to convolve
aquisition_image_path = "/media/ivar/HDD3TB2/IN9400_exercises/Ivar_MRI/FLAIR 3D/DICOM"
image_slices_unsorted = glob(aquisition_image_path + "/IM*")
def get_image_number(file):
    return int(file[len(file)-list(reversed(file)).index("_"):])
image_slices = sorted(image_slices_unsorted, key=get_image_number)

# Swapping and flipping axes to get it right
x_np = np.array([dcmread(s).pixel_array for s in image_slices if np.str(dcmread(s).SeriesNumber) == "1301"])
x_np = np.swapaxes(x_np, 0, 2)
x_np = np.swapaxes(x_np, 0, 1)
x_np = np.flip(x_np, 0)
x_np = np.flip(x_np, 1)

# Copying the data to GPU
x = torch.as_tensor(np.int32(x_np), dtype=torch.double, device=device)
# Important: Mean center and scale by std for x
x -= torch.mean(x)
x /= torch.std(x)

# Convolution kernel/weights .
# A broadcasted 2D gaussian kernel to accomplish smoothing
gaussian = signal.gaussian(gaussian_kernel_width, gaussian_std).reshape((gaussian_kernel_width, 1))
w_np = gaussian*(gaussian.T)*gaussian.reshape((gaussian_kernel_width, 1, 1))
w = torch.as_tensor(w_np, dtype=torch.double, device=device)
# Make shure sum of w equals 1
w -= torch.mean(w)
w /= torch.std(w)

# Bias term
b = torch.randn(1, device=device)
#"""

def show_volume_spimagine(volume_data_pytorch_gpu_tensor, stackUnits=(1, 1, 1)):
    volfig()
    spim_widget = \
    volshow(volume_data_pytorch_gpu_tensor.cpu().numpy(), stackUnits=stackUnits, interpolation='nearest')

def show_volume_spimagine_cpu(numpy_array, stackUnits=(1, 1, 1)):
    volfig()
    spim_widget = \
    volshow(numpy_array, stackUnits=stackUnits, interpolation='nearest')

In [4]:
# Visualizing a slice of the gaussian kernel
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(w[gaussian_kernel_width//2,:,:].cpu().numpy(), cmap="gray")

In [ ]:
x.shape

torch.Size([512, 512, 365])

In [ ]:
# Implement 3D convolution.
import torch.nn as nn

def compute_convolved_image_shape_3D(input_depth, \
                                     input_height, \
                                     input_width, \
                                     kernel_depth, \
                                     kernel_height, \
                                     kernel_width, \
                                     pad_size, \
                                     stride):
    # Formula for the spatial dimensions of an image
    # in an convolutional output layer.
    return torch.as_tensor(np.int(np.floor(1 + (input_depth + 2*pad_size - kernel_depth)/stride)), device=device), \
            torch.as_tensor(np.int(np.floor(1 + (input_height + 2*pad_size - kernel_height)/stride)), device=device), \
            torch.as_tensor(np.int(np.floor(1 + (input_width + 2*pad_size - kernel_width)/stride)), device=device)

def convolution_3D(input_image, weights, pad_size=1, stride=1, save_dir="data"):
    """
    Write a general function to convolve an image with an arbitrary kernel.
    """
    # Flipping weights/kernel to so that it is convolution and not correlation.
    # Did not work with pytorch tensor
    #weights = weights[::-1, ::-1, ::-1]

    # He scaling of weights .
    #weights = torch.div(weights, torch.sqrt(torch.tensor(2/weights.numel())))
    
    
    depth_weights, height_weights, width_weights = weights.shape
    
    depth_input_image, height_input_image, width_input_image = input_image.shape
    
    padding = nn.ConstantPad3d(pad_size, 0)
    
    input_image_padded = padding(input_image)
    #np.pad(input_image, ((pad_size,), (pad_size,), (pad_size,)), mode="constant", constant_values=0)
    
    # Define dimension of output image.
    depth_output_image, height_output_image, width_output_image,  = \
    compute_convolved_image_shape_3D(depth_input_image, \
                                     height_input_image, \
                                     width_input_image, \
                                     depth_weights, \
                                     height_weights, \
                                     width_weights, \
                                     pad_size, \
                                     stride)
    
    output_image = torch.zeros((depth_output_image, \
                                height_output_image, \
                                width_output_image), \
                               device=device)
    
    # Zero padding output image to have equal dimensions
    # as temporary_volume_to_save .
    # Assuming equal padding in all directions.
    #in_out_image_axis_offset = ((input_image_padded.shape[0] - output_image.shape[0])//2)//stride
    
    accumulated_convolved_image = torch.zeros_like(input_image_padded)
    deaccumulated_and_accumulated_convolved_input_image = input_image_padded.clone()
    
    save_number = 1
    iteration = 0
    save_interval = 10
    
    # Firt perform the entire convolution
    for z in range(depth_output_image):
        for y in range(height_output_image):
            for x in range(width_output_image):
                                
                    # The convolution
                    input_image_padded_masked = \
                    input_image_padded[z*stride:z*stride+depth_weights, y*stride:y*stride+height_weights, x*stride:x*stride+width_weights]

                    product = input_image_padded_masked * weights
                    
                    output_image[z, y, x] = \
                    torch.sum(product)
                    
    # Mean normalize the output
    output_image -= torch.mean(output_image)
    output_image /= torch.std(output_image)
    
    # Do the entire convolution again to create the animated sequence with normalized output convolution.
    for z in range(depth_output_image):
        for y in range(height_output_image):
            for x in range(width_output_image):
                
                    iteration += 1
                
                    #if iteration == 20:
                    #    return output_image
                    
                    # Redo the element wise multiplication of the convolution.
                    input_image_padded_masked = \
                    input_image_padded[z*stride:z*stride+depth_weights, y*stride:y*stride+height_weights, x*stride:x*stride+width_weights]
                    
                    product = input_image_padded_masked * weights
                    
                    # Normalize the product according to entire output image.
                    product -= torch.mean(output_image)
                    product /= torch.std(output_image)
                    
                    # THIS WAS NOT NECESSARY, start
                    # Save snapshot of what is left of input image to convolute.
                    #torch.save(deaccumulated_and_accumulated_convolved_input_image, "data/"+str(save_number)+".pt")
                    
                    #save_number += 1
                    # THIS WAS NOT NECESSARY, end
                    
                    # Get the stored convoluted image so far.
                    temporary_volume_to_save \
                    = deaccumulated_and_accumulated_convolved_input_image.clone()
                    
                    # update temporary_volume_to_save with the element.wise multiplicated area.
                    temporary_volume_to_save[z*stride:z*stride+depth_weights, y*stride:y*stride+height_weights, x*stride:x*stride+width_weights]\
                    = product
                    
                    #if iteration % save_interval == 0:
                    # Save temporary_volume_to_save for
                    # 10 intermediate steps for later visualization.
                    if (iteration >= (depth_output_image*height_output_image*width_output_image)//2 + 200 and \
                        iteration <= (depth_output_image*height_output_image*width_output_image)//2 + 210):
                        #"""
                        # Save this view (pre-convolution step, shows the product).
                        # And previous convoluted parts of image
                        torch.save(temporary_volume_to_save, save_dir+"/"+str(save_number)+".pt")
                        save_number += 1
                        #"""
                    
                    # Store the actual convolution output in accumulated_convolved_image .
                    # Using normalized output pixel from previous convolition and
                    # noramlization step (mean centering and divide by std).
                    accumulated_convolved_image[z*stride+depth_weights//2, \
                                                y*stride+height_weights//2, \
                                                x*stride+width_weights//2]\
                    = output_image[z, y, x].clone()
                    
                    # Update temporary_volume_to_save and overwrite
                    # deaccumulated_and_accumulated_convolved_input_image
                    # with a copy of it.
                    temporary_volume_to_save[0:z*stride+depth_weights, \
                                             0:y*stride+height_weights, \
                                             0:x*stride+width_weights] \
                    = accumulated_convolved_image[0:z*stride+depth_weights, \
                                                  0:y*stride+height_weights, \
                                                  0:x*stride+width_weights]
                    
                    deaccumulated_and_accumulated_convolved_input_image \
                    = temporary_volume_to_save.clone()
    
    # Save copies of the convolved image with input image shape.
    num_copies_to_save = 1
    for i in range(num_copies_to_save):
        torch.save(accumulated_convolved_image, save_dir+"/"+str(save_number)+".pt")
        save_number += 1
    
    if stride > 1:
        # Render images for animated high-res -> low-res
        # affine scaling.
        
        print(output_image.shape)
        accumulated_convolved_image_upscaled = \
        torch.zeros_like(output_image, device=device)
        
        # Get the relevant pixels from accumulated_convolved_image .
        for z in range(depth_output_image):
            for y in range(height_output_image):
                for x in range(width_output_image):
                    accumulated_convolved_image_upscaled[z, \
                                                         y, \
                                                         x] \
                    = accumulated_convolved_image[z*stride+depth_weights//2, \
                                                  y*stride+height_weights//2, \
                                                  x*stride+width_weights//2]
        
        # Interpolate accumulated_convolved_image from having 
        # shape like output_image to shape like accumulated_convolved_image .
        
        # Needed to reshape into (mini-batch, channels, depth, heigth, width)
        # for intepolate to understand the volume.
        # Also need to compute the scale factor, based on the formula used in
        # compute_convolved_image_shape_3D .
        #"""
        scale_factor_z = stride*(2*pad_size + depth_input_image)/\
        (stride + depth_input_image + 2*pad_size - depth_weights)

        scale_factor_y = stride*(2*pad_size + height_input_image)/\
        (stride + height_input_image + 2*pad_size - height_weights)

        scale_factor_x = stride*(2*pad_size + width_input_image)/\
        (stride + width_input_image + 2*pad_size - width_weights)
        #"""
        
        accumulated_convolved_image_upscaled = \
        nn.functional.interpolate(torch.reshape(accumulated_convolved_image_upscaled, (1, \
                                                                                      1, \
                                                                                      accumulated_convolved_image_upscaled.shape[0], \
                                                                                       accumulated_convolved_image_upscaled.shape[1], \
                                                                                       accumulated_convolved_image_upscaled.shape[2]
                                                                                      )), \
                                  scale_factor=(scale_factor_z, \
                                                scale_factor_y, \
                                                scale_factor_x), \
                                  mode="nearest")
        """
        # Replication pad the rest of the missing shape size.
        pad_z = \
        (input_image_padded.shape[0] - accumulated_convolved_image_upscaled.shape[2]) // 2
        pad_y = \
        (input_image_padded.shape[1] - accumulated_convolved_image_upscaled.shape[3]) // 2
        pad_x = \
        (input_image_padded.shape[2] - accumulated_convolved_image_upscaled.shape[4]) // 2
        
        padding = nn.ReplicationPad3d((pad_x, pad_x, pad_y, pad_y, pad_z, pad_z))
        accumulated_convolved_image_upscaled = \
        padding(accumulated_convolved_image_upscaled)
        """
        
        #"""
        # Reshape back to (depth, heigth, width) before saving.
        accumulated_convolved_image_upscaled_reshaped = \
        torch.reshape(accumulated_convolved_image_upscaled, \
                      (accumulated_convolved_image_upscaled.shape[2], \
                       accumulated_convolved_image_upscaled.shape[3], \
                       accumulated_convolved_image_upscaled.shape[4]))
        
        # Trick to make shure that the interpolated image has exactly
        # equal dimensions to padded input image
        # using zero padding.
        accumulated_convolved_image_upscaled_reshaped_padded = \
        torch.zeros_like(input_image_padded)
        accumulated_convolved_image_upscaled_reshaped_padded[:accumulated_convolved_image_upscaled_reshaped.shape[0], \
                                                             :accumulated_convolved_image_upscaled_reshaped.shape[1], \
                                                             :accumulated_convolved_image_upscaled_reshaped.shape[2]] \
        = accumulated_convolved_image_upscaled_reshaped
        """
        Not necessary to normalize 
        accumulated_convolved_image_upscaled_reshaped 
        since it is already normalized through the 
        commands above:
        
        # Mean normalize the output
        output_image -= torch.mean(output_image)
        output_image /= torch.std(output_image)
        """
         
        # Save copies of this image.
        for i in range(num_copies_to_save):
        
            torch.save(accumulated_convolved_image_upscaled_reshaped, \
                       save_dir+"/"+str(save_number)+".pt")
            save_number += 1
        
    # Lastly, save the actual output image.
    torch.save(output_image, save_dir+"/"+str(save_number)+".pt")
    save_number += 1
        
    return output_image

In [ ]:
# Perform the convolution
run_output_folder = "mri_all_normalized_nostride"
save_dir = "data/" + run_output_folder

import os
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# From the mri run
#z = convolution_3D(x, w, pad_size=30, stride=50, save_dir=save_dir)# + b

# From the mri_all_normalized run
#z = convolution_3D(x, w, pad_size=30, stride=25, save_dir=save_dir)# + b

# From the mri_all_normalized_nostride run
z = convolution_3D(x, w, pad_size=30, stride=1, save_dir=save_dir)# + b

#print(x.shape)
#print(w.shape)
#print(z.shape)

In [ ]:
#show_volume_spimagine(x)
#show_volume_spimagine(w)
#show_volume_spimagine(y)

In [ ]:
#torch.cuda.empty_cache()

#x_fft = torch.rfft(x, signal_ndim=3)

In [ ]:
"""

# Load stored timeseries data of the convolution.
import numpy as np
import torch

run_output_folder = "mri_all_normalized_nostride"
save_dir = "data/" + run_output_folder

from glob import glob

aquisition_image_path = "data/" + run_output_folder
images_unsorted = glob(aquisition_image_path + "/*.pt*")
def get_image_number(file):
    return int(file[len(save_dir + "/"):-len(".pt")])
images = sorted(images_unsorted, key=get_image_number)

#selected_image_index = len(images) // 2
#image_selection = [images[0]] + [images[len(images)//2]] + [images[-2]] + [images[-1]]
#image_selection = [images[-(501 + 501)]] + [images[-501]]
#image_selection = [images[selected_image_index + i] for i in range(3)]
#image_selection = [images[-3]] + [images[-2]]
image_selection = [images[-4]]

#z_series = np.array([torch.load(image).cpu().numpy() for image in image_selection])

"""

In [ ]:
#[z.shape for z in z_series]

In [ ]:
#show_volume_spimagine_cpu(z_series, stackUnits=(0.5, 0.48828125, 0.48828125))
# render pngs with spimagine.
# create gif with
# ffmpeg -i output_%03d.png -filter:v "setpts=5.0*PTS" rendering_5x_slow_vol_35_gauss_w_22_std_12_all_normalized.gif

In [ ]:
#images[:-4]

In [ ]:
"""
# Convert torch pt no numpy npy format
numpy_save_dir = save_dir + "/" + "npy"

import os
if not os.path.exists(numpy_save_dir):
    os.makedirs(numpy_save_dir)
    
for i, image in enumerate(images[:-4]):
    np.save(numpy_save_dir + "/" + str(i+1), torch.load(image).cpu().numpy())
"""